# **SQL Server Database Mail**

- Carlos Eduardo Gimenes
- Last Modified: July, 2023

> **Copyright (C) 2023 Carlos Eduardo Gimenes**  
> All rights reserved.  
> You may alter this code for your own _non-commercial_ purposes.  
> You may republish altered code as long as you include this copyright and give due credit.
> 
> THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE IMPLIED WARRANTIES OF MERCHANTABILITY AND/OR FITNESS FOR A PARTICULAR PURPOSE.

## <span style="color: var(--vscode-foreground);"><b>Database Mail and Email Alerts with SQL Agent on Linux</b></span>

[Fonte: Microsoft learn](https://learn.microsoft.com/en-us/sql/linux/sql-server-linux-db-mail-sql-agent?view=sql-server-ver16)<span style="color: var(--vscode-foreground);"><b><br></b></span>

### **Step 1 - Enable Database Mail**

In [ ]:
USE master 
GO 
sp_configure 'show advanced options',1 
GO 
RECONFIGURE WITH OVERRIDE 
GO 
sp_configure 'Database Mail XPs', 1 
GO 
RECONFIGURE  
GO

### **Step 2 - Create a new account**

In [ ]:
EXECUTE msdb.dbo.sysmail_add_account_sp 
@account_name = 'MSSQLServer',                                -- ATENCAO AJUSTAR (Sugestao Documentacao: SQLAlerts)
@description = 'Account for Automated DBA Notifications',     -- ATENCAO AJUSTAR (Sugestao Documentacao: Account for Automated DBA Notifications)
@email_address = 'usuario@dominio.com',                       -- ATENCAO AJUSTAR
@replyto_address = 'usuario@dominio.com',                     -- ATENCAO AJUSTAR
@display_name = 'NomeServidor - SQL Agent',                   -- ATENCAO AJUSTAR (Sugestao Documentacao: SQL Agent)
@mailserver_name = 'smtp.gmail.com',                          -- ATENCAO AJUSTAR (Este exemplo utiliza uma conta do Gmail)
@port = 587,                                                  -- ATENCAO AJUSTAR (Este exemplo utiliza porta do Gmail)
@enable_ssl = 1, 
@username = 'usuario@gmail.com',                              -- ATENCAO AJUSTAR (Nome da conta do Gmail para o mailserver informado acima)
@password = 'INFORMAR-SENHA-APLICATIVO'                       -- ATENCAO AJUSTAR (Necessario gerar para utilizar aqui)

GO

### **Step 3 - Create a default profile**

In [ ]:
EXECUTE msdb.dbo.sysmail_add_profile_sp 
@profile_name = 'MSSQLServer',                                      -- ATENCAO AJUSTAR (Sugestao Documentacao: default)
@description = 'Profile for sending Automated DBA Notifications'    -- ATENCAO AJUSTAR (Sugestao Documentacao: Profile for sending Automated DBA Notifications)
GO

### **Step 4 - Add the Database Mail account to a Database Mail profile**

In [ ]:
EXECUTE msdb.dbo.sysmail_add_principalprofile_sp 
@profile_name = 'MSSQLServer',                              -- ATENCAO AJUSTAR (Sugestao Documentacao: default)
@principal_name = 'public',                                 -- ATENCAO AJUSTAR (Sugestao Documentacao: public)
@is_default = 1 ;

### **Step 5 - Add account to profile**

In [ ]:
EXECUTE msdb.dbo.sysmail_add_profileaccount_sp   
@profile_name = 'MSSQLServer',                                  -- ATENCAO AJUSTAR (Sugestao Documentacao: default)
@account_name = 'MSSQLServer',                                  -- ATENCAO AJUSTAR (Sugestao Documentacao: SQLAlerts)
@sequence_number = 1;

### **Step 6 - Send test email and Check Status**

- **Send test email**

In [ ]:
EXECUTE msdb.dbo.sp_send_dbmail 
@profile_name = 'MSSQLServer', 
@recipients = 'usuario@dominio.com',            -- ATENCAO: AJUSTAR
@Subject = 'Testing DBMail', 
@Body = 'This message is a test for DBMail' 
GO

- **Check E-mails not send**

In [ ]:
Select 
* 
from 
msdb.dbo.sysmail_unsentitems 
Order by
send_request_date desc

- **Check E-mails send**
    - **Sent\_Status**
        - **0 - unsent**
        - **1 - sent**
        - **2 - failed (default)**
        - **3 - retrying**

In [ ]:
Select 
Top 5
sent_status,
sent_date,
send_request_date,
send_request_user
from 
msdb.dbo.sysmail_mailitems
Order by
send_request_date desc

### **Step 7 - Set Database Mail Profile using mssql-conf or environment variable**

- <span style="font-size:14px;">You can use the mssql-conf utility or environment variables to register your DB Mail profile. In this case, let's call our profile default.</span>

In [ ]:
# via mssql-conf
sudo /opt/mssql/bin/mssql-conf set sqlagent.databasemailprofile MSSQLServer
# via environment variable
MSSQL_AGENT_EMAIL_PROFILE=MSSQLServer

### **Step 8 - Set up an operator for SQLAgent job notifications**

In [ ]:
EXEC msdb.dbo.sp_add_operator 
@name=N'Alerta_BD',                                         -- ATENCAO AJUSTAR (Sugestao Documentacao: JobAdmins)
@enabled=1, 
@email_address=N'usuario@dominio.com',                      -- ATENCAO: AJUSTAR
@category_name=N'[Uncategorized]'                           -- ATENCAO AJUSTAR (Sugestao Documentacao: Uncategorized)
GO

### **Step 9 - Create a sample database**

- This database is used for the daily backup job

In [ ]:
CREATE DATABASE SampleDB;

### **Step 10 - Create a job named** **Daily SampleDB Backup**

- <span style="font-size: 14px;"><b>Use sp_add_job to create a job named Daily SampleDB Backup</b></span>

In [ ]:
-- Adds a new job executed by the SQLServerAgent service
-- called 'Daily SampleDB Backup'
USE msdb ;
GO
EXEC dbo.sp_add_job
   @job_name = N'Daily SampleDB Backup' ;
GO

- <span style="font-size: 14px;"><b>Call sp_add_jobstep to create a job step that creates a backup of the SampleDB database</b></span>

In [ ]:
-- Adds a step (operation) to the job
EXEC sp_add_jobstep
   @job_name = N'Daily SampleDB Backup',
   @step_name = N'Backup database',
   @subsystem = N'TSQL',
   @command = N'BACKUP DATABASE SampleDB TO DISK = \
      N''/arquivos3/mssql/backup/completo/SampleDB.bak'' WITH NOFORMAT, NOINIT, \
      NAME = ''SampleDB-full'', SKIP, NOREWIND, NOUNLOAD, STATS = 10',
   @retry_attempts = 5,
   @retry_interval = 5 ;
GO

- <span style="font-size: 14px;"><b>Then create a daily schedule for your job with sp_add_schedule</b></span>

In [ ]:
-- Creates a schedule called 'Daily'
EXEC dbo.sp_add_schedule
   @schedule_name = N'Daily SampleDB',
   @freq_type = 4,
   @freq_interval = 1,
   @active_start_time = 233000 ;
USE msdb ;
GO

- <span style="font-size: 14px;"><b>Attach the job schedule to the job with sp_attach_schedule.</b></span>

In [ ]:
-- Sets the 'Daily' schedule to the 'Daily SampleDB Backup' Job
EXEC sp_attach_schedule
   @job_name = N'Daily SampleDB Backup',
   @schedule_name = N'Daily SampleDB';
GO

- <span style="font-size: 14px;"><b>Use sp_add_jobserver to assign the job to a target server. In this example, the target is the local server</b></span>

In [ ]:
EXEC dbo.sp_add_jobserver
   @job_name = N'Daily SampleDB Backup',
   @server_name = N'(LOCAL)';
GO

- <span style="font-size: 14px;"><b>Start the job with sp_start_job</b></span>

In [ ]:
EXEC dbo.sp_start_job N' Daily SampleDB Backup' ;
GO

### **Setp 11 - Send email when '****Daily SampleDB Backup****' succeeds**

In [ ]:
EXEC msdb.dbo.sp_update_job 
@job_name='Daily SampleDB Backup', 
@notify_level_email=3, 
@notify_email_operator_name=N'Alerta_BD'            -- ATENCAO AJUSTAR (Sugestao Documentacao: JobAdmins)
GO

### **Step 12 - After tests Delete Database & Job Test**

In [ ]:
USE [msdb]
GO

EXEC msdb.dbo.sp_delete_job @job_name = N'Daily SampleDB Backup'
GO